In [35]:
# imports pandas, a package for data manipulation and analysis
import pandas as pd  
import pyodbc

In [36]:
# imports the file downloaded using Youtube API

df = pd.read_csv ('youtube_data.csv', index_col=0)
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,lH7HfwUFnYA,Pivot Tables in Excel | Excel Tutorials for Be...,2021-12-14,3516,240,41
1,zOR0-nygfDE,Tableau Tutorial for Beginners | Full Project ...,2021-12-07,5936,148,26
2,A4SVUF-fTwc,Tableau Tutorial for Beginners | Joins | Part 4/5,2021-11-23,5220,152,14
3,FnTunxd8tAM,Top 5 Major Red Flags in Job Descriptions,2021-11-18,9128,555,58
4,-_z_aphWSEo,Truth About My Job (Song) - As a Data Analyst,2021-11-08,6870,647,90


In [37]:
# to establish a connection with DB

def connect_to_db (server_name, dbname):
    try:
        cnxn = pyodbc.connect(
            "driver={ODBC Driver 17 for SQL Server}; server='serve_name'; database= 'database_name' ; trusted_connection=YES;")

    except pyodbc.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return cnxn
    

# function to a new table in SQL database    
    
def create_table(curr):
    table_columns = ("""CREATE TABLE 'new_table_name' (
                                        video_id VARCHAR(255) PRIMARY KEY NOT NULL,
                                        video_title TEXT NOT NULL,
                                        upload_date DATE NOT NULL,
                                        view_count INTEGER NOT NULL,
                                        like_count INTEGER NOT NULL
                                        )""")
    curr.execute(table_columns)
    

# to insert a new video into the table   
    
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count):
    insert_into_videos = ("""INSERT INTO 'new_table_name'( video_id, video_title, upload_date, view_count, like_count)
                    VALUES(?,?,?,?,?);""")
    
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count)
    
    curr.execute(insert_into_videos, row_to_insert)
    

# to update the video information after data refresh      
    
def update_row(curr, video_id, video_title, upload_date, view_count, like_count):
    query = ("""UPDATE 'new_table_name'
            SET video_title = ?,
                upload_date = ?,
                view_count = ?,
                like_count = ?
                            
            WHERE video_id = ?;""")
    
    vars_to_update = (video_title, upload_date, view_count, like_count, video_id)
    curr.execute(query, vars_to_update)
    

# to check if the video exists using primary key    
    
def check_if_video_exists(curr, video_id):
    query = ("""SELECT video_id FROM 'new_table_name' WHERE video_id = ?""")
    
    curr.execute(query,(video_id))    
    return curr.fetchone() is not None

# to add a new video into table

def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'])
        
# create a temporary dataframe 
        
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count'])
    
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']):
            update_row(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'])
        else:
            tmp_df = tmp_df.append(row)
            
    return tmp_df

In [38]:
#database credentials

server_name = 'serve_name'
dbname = 'database_name'

#establish a connection with db

cnxn = connect_to_db (server_name, dbname)
curr = cnxn.cursor()

Connected!


In [39]:
#create_table

create_table(curr)
cnxn.commit()

In [40]:
#update data to existing videos

new_vid_df = update_db(curr,df)
cnxn.commit()

In [41]:
#insert new video into db

append_from_df_to_db(curr, new_vid_df)
cnxn.commit()

In [42]:
#view db data

curr.execute('SELECT * FROM 'new_table_name')
print(curr.fetchall())

[('-_z_aphWSEo', 'Truth About My Job (Song) - As a Data Analyst', datetime.date(2021, 11, 8), 6870, 647), ('-Inc1be4zSg', 'The Hiring Process from an Interviewer&#39;s Perspective | Alex The Analyst Show | Episode 1', datetime.date(2020, 9, 3), 7826, 338), ('_gP-DH0iZLw', 'Top 5 Reasons Data Analytics is a Good Career Choice', datetime.date(2020, 9, 8), 17390, 671), ('_zxPx1PQCqI', 'I Am No Longer a Data Analyst | Turning Down PwC | New Job', datetime.date(2021, 8, 11), 91549, 3306), ('0HRgbb1PcDQ', 'Data Analyst Salary (Expectations, Bonuses, 401K, ESPP) | Alex The Analyst Show | Episode 13', datetime.date(2020, 12, 3), 11607, 448), ('1gdKC5O0Pwc', 'R vs Python | Which is Better for Data Analysis?', datetime.date(2021, 2, 16), 73800, 2033), ('1xT68oeQTd0', 'Data Analyst Expectations vs Reality', datetime.date(2020, 10, 6), 71112, 2688), ('465x3JmMJYU', 'How NOT to Become a Data Analyst (Funny?)', datetime.date(2021, 9, 7), 13545, 616), ('4rfr6A3lO-Y', 'Data Analyst Resume | Reviewing 